In [1]:
%python

from watson_developer_cloud.natural_language_understanding_v1 import *
import re
from pyspark.sql.types import *

class WatsonUnderstandingWrapper:
  
  #Change according to Watson API
  LAST_VERSION='2018-03-16'
  
  def __init__(self, user,passwd):
        self.user = user
        self.password =passwd
        self.nlu=NaturalLanguageUnderstandingV1(version=self.LAST_VERSION,  username=self.user,password=self.password)
  
  def printCategories(self,texto):  
      response=self.nlu.analyze(text=texto,features=Features(categories=CategoriesOptions()),language='es')
      return response['categories'][0] if len(response['categories'])>0 else None
  
  def printSemantics(self,texto):
      response=self.nlu.analyze(text=texto,features=Features(    semantic_roles=SemanticRolesOptions(entities=True)),language='es')
      return response['semantic_roles']if len(response['semantic_roles'])>0 else None
    
  def printSentiment(self,texto,target):
      response=self.nlu.analyze(text=texto, features=Features(sentiment=SentimentOptions(targets=['angelamrobledo'])),language='es')
      return response['sentiment']if len(response['sentiment'])>0 else None
    
  def printEntities(self,texto):
      response=self.nlu.analyze(text=texto, features=Features(entities=EntitiesOptions(sentiment=True,emotion=True)),language='es')
      return response['entities']if len(response['entities'])>0 else None  
  
  schemanaly = StructType([
                StructField('score'     , FloatType()   , False),
                StructField('label'  , StringType()    , False)
            ])
  
  
  schemasem = ArrayType(StructType([
                StructField('subject' , StructType([ StructField('text'  , StringType()    , True)])   , True),
                StructField('sentence', StringType()    , True),
                StructField('object'  , StructType([ StructField('text'  , StringType()    , True)])   , True),    
                StructField('action', StructType([StructField('verb', StructType([StructField('text', StringType(), True),StructField('tense', StringType(), True)]),True),StructField('text', StringType(), True),StructField('normalized', StringType(), True)]),True)  
]))
  
  #[{'type': 'TwitterHandle', 'text': '@angelamrobledo', 'relevance': 0.978347, 'count': 1, 'sentiment': {'label': 'negative', 'score': -0.91139}}]
  schemaent = ArrayType(StructType([
      StructField('type',StringType(),True),
      StructField('text',StringType(),True),
      StructField('relevance',StringType(),True),
      StructField('count',LongType(),True),
      StructField('sentiment',StructType([StructField('label',StringType(),True),StructField('score',FloatType(),True)]),True)
  ])
  )
  

In [2]:
describe table debatevice

In [3]:
select user  `@ twitter`,count(*) tweets from(
select explode(entities.user_mentions.screen_name) user,case when retweeted_status is null then 'No' else 'Si' end Es_RT	from debatevice where  size(entities.user_mentions)>0

)
where user in ('angelamrobledo','mluciaramirez','ClaudiaLopez','PinzonBueno','ClaraLopezObre')
group by user
order by 2 desc

In [4]:
SELECT case t.text when '@ClaudiaLopez' then 'Claudia L.' when '@mluciaramirez' then 'Marta L.' when '@angelamrobledo' then 'Ángela R.'  when '@PinzonBueno' then 'Pinzón' when '@ClaraLopezObre' then 'Clara L.'else 'otro' end entity,window(CAST (from_unixtime(unix_timestamp(created_at,'EEE MMM d HH:mm:ss Z yyyy'))AS TIMESTAMP) ,"5 minutes").start fecha,sum(t.sentiment.score*t.relevance) sum_score
--SELECT CAST(T.SENTIMENT.RELEVANC AS LONG)
FROM(
select ID,EXPLODE(SENTIMENT) T from vicentiment
where array_contains(sentiment.type,'Person') 
OR ARRAY_CONTAINS(sentiment.type,'TwitterHandle')
  ) Q1 JOIN DEBATEVICE D ON Q1.ID=D.ID
  WHERE    T.relevance>=0.6 AND from_unixtime(unix_timestamp(created_at,'EEE MMM d HH:mm:ss Z yyyy')-18000)>='2018-05-18 22:00:00'
  AND from_unixtime(unix_timestamp(created_at,'EEE MMM d HH:mm:ss Z yyyy')-18000)<='2018-05-19 00:35:00'
  AND  T.TYPE ='TwitterHandle' AND SUBSTRING(T.TEXT,2,LENGTH(T.TEXT)-1) in ('angelamrobledo','mluciaramirez','ClaudiaLopez','PinzonBueno','ClaraLopezObre')
  --group by from_unixtime(unix_timestamp(created_at,'EEE MMM d HH:mm:00 Z yyyy')-18000)
  group by t.text,window(CAST (from_unixtime(unix_timestamp(created_at,'EEE MMM d HH:mm:ss Z yyyy'))AS TIMESTAMP) ,"5 minutes")
  ORDER BY 2